### Условия
- 1.1 - случайная генерация начальной популяции
- 2.1 - выбор особи пропорциональоно приспособленности
- 3.1 - скрещивание с тремя точками
- 4.3 - мутация с добавлением одной случайной вещи
- 5.3 - новая популяция формируется с заменой родителей

In [203]:
import random as rnd
import numpy as np
import json

### Чтение данных

In [204]:
file = open('30.txt')        
first_line = file.readline().split(' ')
data = {'items': []}
max_weight = int(first_line[0])
max_volume = float(first_line[1])
for line in file:
    values = [x for x in line.split(' ')]
    data['items'].append({'weight': int(values[0]), 'volume': float(values[1]), 'price': int(values[2])})
data

{'items': [{'weight': 708, 'volume': 1.1, 'price': 244},
  {'weight': 342, 'volume': 0.7, 'price': 307},
  {'weight': 241, 'volume': 0.9, 'price': 385},
  {'weight': 384, 'volume': 0.5, 'price': 126},
  {'weight': 1227, 'volume': 0.7, 'price': 191},
  {'weight': 1449, 'volume': 0.9, 'price': 296},
  {'weight': 877, 'volume': 0.6, 'price': 260},
  {'weight': 334, 'volume': 1.0, 'price': 146},
  {'weight': 1056, 'volume': 1.0, 'price': 142},
  {'weight': 280, 'volume': 0.9, 'price': 358},
  {'weight': 1179, 'volume': 0.7, 'price': 145},
  {'weight': 1594, 'volume': 0.6, 'price': 203},
  {'weight': 353, 'volume': 1.2, 'price': 231},
  {'weight': 1178, 'volume': 0.7, 'price': 282},
  {'weight': 470, 'volume': 1.0, 'price': 317},
  {'weight': 913, 'volume': 0.8, 'price': 275},
  {'weight': 908, 'volume': 0.6, 'price': 294},
  {'weight': 528, 'volume': 1.1, 'price': 247},
  {'weight': 1638, 'volume': 0.9, 'price': 301},
  {'weight': 730, 'volume': 1.1, 'price': 388},
  {'weight': 624, 'volum

In [205]:
print('Грузоподъемность: '+ str(max_weight) + '\nВместимость: '+ str(max_volume))

Грузоподъемность: 13000
Вместимость: 12.0


In [206]:
items_count = len(data['items'])
population_size = 200
populations_count = 500
cheapest_item = sorted(data['items'], key=lambda dct: dct['price'])[0]['price']

### Создание популяции

In [207]:
def create_individual():
    individual = [rnd.randint(0, 1) for x in range(items_count)]
    while not (check(individual)):
        individual = [rnd.randint(0, 1) for x in range(items_count)]
    return individual

def check(individual):
    total_weight = 0
    total_volume = 0
    for i in range(0, 30):
        if individual[i] != 0:
            total_weight += data['items'][i]['weight']
            total_volume += data['items'][i]['volume']
    if (total_weight > max_weight or total_volume > max_volume):
        return False
    else: return True
    
def create_population():
    population = [create_individual() for x in range(population_size)]
    return population

### Функция приспособления

In [208]:
def fitness(individual, data):
    weight, volume, price = 0, 0, 0
    data
    for (selected, item) in zip(individual, data):
        if selected:
            weight += item['weight']
            volume += item['volume']
            price += item['price']
    if weight > max_weight or volume > max_volume:
        price = 0
    return price

### Отбор особей для скрещивания

In [209]:
def choose_individuals(population, items):
    chosen_individuals = []
    candidates = []
    for i in range(len(population)):
        candidates.append(fitness(population[i], items))
    best_fitness = 0
    worst_fitness = candidates[0]
    
    for j in range(len(candidates)):
        if candidates[j] > best_fitness:
            best_fitness = candidates[j]

    if candidates[j] < worst_fitness and candidates[j] != 0:
        worst_fitness = candidates[j]
        
    difference = best_fitness - worst_fitness        
    for q in range(len(candidates)):
        candidates[q] = (candidates[q] - worst_fitness) / difference
        if rnd.random() < candidates[q]:
            chosen_individuals.append(population[q])
    return chosen_individuals        

### Скрещивание

In [210]:
def get_children(first_parent, second_parent, items):
    point1 = rnd.randint(0, items_count/3)
    point2 = rnd.randint(point1, items_count/3 * 2)
    point3 = rnd.randint(point2, items_count)

    first_child = first_parent[0:point1] + second_parent[point1:point2] + first_parent[point2:point3] + second_parent[point3:]
    second_child = second_parent[0:point1] + first_parent[point1:point2] + second_parent[point2:point3] + first_parent[point3:]
    
    if fitness(first_child, items) == 0:
        firt_child = first_parent
    if fitness(second_child, items) == 0:
        second_child = second_parent
        
    return first_child, second_child

def crossing_over(parents, items):
    children = []
    while len(parents) > 1:
        first_parent = rnd.choice(parents)
        parents.remove(first_parent)
        second_parent = rnd.choice(parents)
        parents.remove(second_parent)
        children.extend(get_children(first_parent, second_parent, items))
    children.extend(parents)
    return children

### Мутация

In [211]:
def mutation(individual):
    item_to_add = rnd.randint(0, len(individual) - 1)
    mutated_individual = individual.copy()
    while individual[item_to_add] != 0:
        item_to_add = rnd.randint(0, len(individual) - 1)
    mutated_individual[item_to_add] = 1
    return mutated_individual

def mutation_for_population(population):
    number_for_mutation = int(len(population) * 0.05)
    for i in range(number_for_mutation):
        individual_for_mutation = rnd.randint(0, len(population) - 1)
        mutation(population[individual_for_mutation])
    return population

### Формирование новой популяции

In [212]:
def create_new_population(population, parents, children):
    new_population = []
    for ind in population:
        if ind not in parents:
            new_population.append(ind)
    new_population.extend(children)
    return new_population

### Поиск лучшей особи

In [213]:
def get_best_individual(population, items):
    candidates = []

    for ind in population:
        candidates.append((fitness(ind, items), ind))

    best_ind = candidates[0]
    for candidate in candidates:
        best_ind = candidate if candidate[0] > best_ind[0] else best_ind

    return best_ind

### Генетический алгоритм

In [214]:
def ga(population, items, max_weight, max_volume, cheapest_item):
    best_values = []
    
    for gen in range(populations_count):
        
        parents = choose_individuals(population, items)
        children = crossing_over(parents, items)
        population = create_new_population(population, parents, children)
        mutation_for_population(population)
        
        while len(population) < population_size:
            population.append(create_individual())
            
        best_individual = get_best_individual(population, items)
        best_ind_value = best_individual[0]
        best_values.append(best_ind_value)
        
        if abs(min(best_values) - max(best_values)) <= cheapest_item:
            break
            
    return best_individual

In [228]:
pop = create_population()
result = ga(pop, data['items'], max_weight, max_volume, cheapest_item)
print(result)

(4348, [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1])


In [176]:
def get_description(individual, items):
    weight, volume = 0, 0
    for (selected, item) in zip(individual, items):
        if selected:
            weight += item['weight']
            volume += item['volume']
    return weight, volume

desc = get_description(result[1], data['items'])

In [200]:
results = {
    'Best result' : str(result[1]),
    'Value' : result[0],
    'Weight' : desc[0],
    'Volume' : desc[1]    
}

In [201]:
with open('4.2_res.jsn', 'w') as file:
    json.dump(results, file, indent=10)